In [2]:
import ijson
from decimal import Decimal
import json
import pandas as pd

In [1]:
path_local = 'C:\\Users\\denis.iglesias\\OneDrive - HESSO\\03 Master\\01 Cours\\12 TSM-AdvDaBa\\02 Labo\\03 Labo 2.2 neo4j large database\\'
input_file_path = path_local + 'temp_file.json'
output_file_path = path_local + 'output_newline_delimited_json.json'
path_parquet = path_local + 'output_parquet.parquet'

In [ ]:
def extract_required_data(item):
    return {
        '_id': item.get('_id', None),
        'title': item.get('title', None),
        'authors': item.get('authors', []),
        'references': item.get('references', [])
    }

def convert_decimal(obj):
    if isinstance(obj, Decimal):
        return float(obj)
    return obj

with open(input_file_path, 'rb') as input_file, open(output_file_path, 'w', encoding='utf-8') as output_file:
    for item in ijson.items(input_file, 'item'):
        formatted_item = extract_required_data(item)
        json.dump({k: convert_decimal(v) for k, v in formatted_item.items()}, output_file)
        output_file.write('\n')

In [5]:
df = pd.read_json(output_file_path, lines=True)

def modify_authors(authors):
    if not authors:  # Check if authors list is empty
        return None
    authors_list = [{'_id': author.get('_id', None), 'name': author.get('name', None)} 
                    for author in authors if 'name' in author and '_id' in author]
    return authors_list if authors_list else None

def modify_references(references):
    return references if references else None

# Modify authors and references columns
df['authors'] = df['authors'].apply(modify_authors)
df['references'] = df['references'].apply(modify_references)

df.to_parquet(path_parquet, engine='pyarrow', compression='snappy')

In [21]:
# Reading the file into a LazyFrame
df = pd.read_parquet(path_parquet, engine='pyarrow')

In [4]:
print('Number of unique _id : ', df['_id'].nunique())
print('Number of duplicate _id : ', df.duplicated(subset=['_id']).sum())
print('Number of duplicate title : ', df.duplicated(subset=['title']).sum())

Number of unique _id :  5069313
Number of duplicate _id :  0
Number of duplicate title :  40755
